In [ ]:
pip install pycaret==3.3.2 m2cgen pandas numpy scikit-learn

In [11]:
import pandas as pd
from pycaret.classification import *
import m2cgen as m2c
import numpy as np

In [12]:
# Simula 100 amostras de dois tipos de movimento (classe1 e classe2)
n_samples = 100
data = []

for _ in range(n_samples):
    # Classe 1: sinal mais "calmo"
    signal = np.random.normal(loc=0.0, scale=0.1, size=50)
    features = [
        np.mean(signal),         # mean
        np.std(signal),          # std
        np.sqrt(np.mean(signal**2)),  # rms
        np.ptp(signal)           # peak-to-peak
    ]
    features.append("classe1")
    data.append(features)

for _ in range(n_samples):
    # Classe 2: sinal mais "agitado"
    signal = np.random.normal(loc=0.0, scale=0.3, size=50)
    features = [
        np.mean(signal),
        np.std(signal),
        np.sqrt(np.mean(signal**2)),
        np.ptp(signal)
    ]
    features.append("classe2")
    data.append(features)

# Cria DataFrame
df = pd.DataFrame(data, columns=["mean","std","rms","ptp","label"])

# Salva CSV
df.to_csv("dataset_features.csv", index=False)
print("CSV gerado!")

CSV gerado!


In [14]:
df = pd.read_csv("dataset_features.csv") # colunas: mean,std,rms,ptp,label
features = ["mean","std","rms","ptp"]
target = "label"
s = setup(data=df[features+[target]], target=target,
 session_id=42, normalize=True, fold=5, verbose=False)

# 1) Decision Tree (leve)
dt = create_model("dt") # árvore padrão
dt_final = finalize_model(dt)
# Extract the scikit-learn model from the PyCaret pipeline
dt_model = dt_final.steps[-1][1]
open("model_dt.c","w").write(m2c.export_to_c(dt_model))

# 2) SVM linear (leve e bom p/ MCU)
svm = create_model("svm") # linear por padrão no PyCaret (scikit SVC linear kernel='rbf'? No PyCaret "svm" costuma ser LinearSVM/LinearSVC)
svm_final = finalize_model(svm)
# Extract the scikit-learn model from the PyCaret pipeline
svm_model = svm_final.steps[-1][1]
open("model_svm.c","w").write(m2c.export_to_c(svm_model))
print("Gerados: model_dt.c e model_svm.c")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Mean,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Std,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Mean,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Std,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Gerados: model_dt.c e model_svm.c


In [15]:
from google.colab import files

files.download("model_dt.c")
files.download("model_svm.c")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>